In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import random

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.uint8)
x_test = x_test.astype(np.uint8)

def generate_keys(img_shape=(28, 28), perm_block_size=1):
    h, w = img_shape[0] // perm_block_size, img_shape[1] // perm_block_size
    num_perm_blocks = h * w
    perm_key = list(range(num_perm_blocks))
    random.shuffle(perm_key)
    aes_key = get_random_bytes(16)
    return aes_key, perm_key

def encrypt_and_permute_images(images, aes_key, perm_key, aes_block_size=4, perm_block_size=1):
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_images = []

    for img in images:
        # Step 1: Encrypt using 4x4 AES blocks
        encrypted_img = np.zeros((28, 28), dtype=np.uint8)
        for i in range(0, 28, aes_block_size):
            for j in range(0, 28, aes_block_size):
                block = img[i:i+aes_block_size, j:j+aes_block_size].flatten()
                encrypted = cipher.encrypt(bytes(block))
                enc_block = np.frombuffer(encrypted, dtype=np.uint8).reshape((aes_block_size, aes_block_size))
                encrypted_img[i:i+aes_block_size, j:j+aes_block_size] = enc_block

        # Step 2: Permute using 2x2 blocks
        blocks = []
        for i in range(0, 28, perm_block_size):
            for j in range(0, 28, perm_block_size):
                blocks.append(encrypted_img[i:i+perm_block_size, j:j+perm_block_size].copy())
        
        permuted_blocks = [blocks[i] for i in perm_key]
        permuted_img = np.zeros((28, 28), dtype=np.uint8)
        idx = 0
        for i in range(0, 28, perm_block_size):
            for j in range(0, 28, perm_block_size):
                permuted_img[i:i+perm_block_size, j:j+perm_block_size] = permuted_blocks[idx]
                idx += 1
        
        encrypted_images.append(permuted_img)

    return np.array(encrypted_images, dtype=np.uint8)


2025-08-07 08:24:57.299700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754555097.553948      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754555097.631001      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
aes_key, perm_key = generate_keys()
x_train_enc = encrypt_and_permute_images(x_train, aes_key, perm_key)
x_test_enc = encrypt_and_permute_images(x_test, aes_key, perm_key)

In [4]:
len(perm_key)

784

##  Predicting using Maximum Similarity Score obtained for each label

In [5]:
from collections import defaultdict
import random
from tqdm import tqdm
import datetime
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

def sample_train_per_label(x_train_enc, y_train, num_samples=6500):
    label_to_images = defaultdict(list)
    for img, label in zip(x_train_enc, y_train):
        if len(label_to_images[label]) < num_samples:
            label_to_images[label].append(img)
        if all(len(imgs) == num_samples for imgs in label_to_images.values()):
            break
    x_train_sampled = []
    y_train_sampled = []
    for label in range(10):
        x_train_sampled.extend(label_to_images[label])
        y_train_sampled.extend([label] * num_samples)
    return np.array(x_train_sampled), np.array(y_train_sampled)

# Prediction logic
def predict_label(test_img, x_train_small, y_train_small):
    scores = defaultdict(list)
    for train_img, label in zip(x_train_small, y_train_small):
        match = np.sum(test_img == train_img)
        scores[label].append(match / 784.0)
    avg_scores = {label: np.mean(score_list) for label, score_list in scores.items()}
    return max(avg_scores, key=avg_scores.get)
    
# Wrap predict_label to allow use in executor (requires args to be passed)
def predict_label_wrapper(args):
    test_img, x_train_small, y_train_small = args
    return predict_label(test_img, x_train_small, y_train_small)

# Sample 1000 images per label
x_train_small, y_train_small = sample_train_per_label(x_train_enc, y_train, num_samples=6500)

# Shuffle test images
perm = np.random.permutation(len(x_test_enc))
x_test_enc = x_test_enc[perm]
y_test = y_test[perm]

args_list = [(img, x_train_small, y_train_small) for img in x_test_enc]
print("Date * Time: ", datetime.datetime.now(), "\nTimestamp: ", datetime.datetime.now())
# Use 4 workers for parallel prediction
with ProcessPoolExecutor(max_workers=4) as executor:
    preds = list(tqdm(executor.map(predict_label_wrapper, args_list), total=len(x_test_enc), desc="Predicting for 10000 MNIST encrypted-permuted Test images...."))
print("Date * Time: ", datetime.datetime.now(), "\nTimestamp: ", datetime.datetime.now())

acc = np.mean(np.array(preds) == y_test) * 100
print(f"Accuracy: {acc:.2f}%")

Date * Time:  2025-08-07 08:27:11.671792 
Timestamp:  2025-08-07 08:27:11.671797


Predicting for 10000 MNIST encrypted-permuted Test images....: 100%|██████████| 10000/10000 [1:05:52<00:00,  2.53it/s]

Date * Time:  2025-08-07 09:33:05.946302 
Timestamp:  2025-08-07 09:33:05.946306
Accuracy: 12.77%


## Predicting using Average Similarity Score Obtained for each label (performs significantly better than random guess)

In [6]:
from collections import defaultdict
import random
from tqdm import tqdm
import datetime
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

# Sample 1000 images per label
x_train_small, y_train_small = sample_train_per_label(x_train_enc, y_train, num_samples=6500)

# Shuffle test images
perm = np.random.permutation(len(x_test_enc))
x_test_enc = x_test_enc[perm]
y_test = y_test[perm]

def sample_train_per_label(x_train_enc, y_train, num_samples=6500):
    label_to_images = defaultdict(list)
    for img, label in zip(x_train_enc, y_train):
        if len(label_to_images[label]) < num_samples:
            label_to_images[label].append(img)
        if all(len(imgs) == num_samples for imgs in label_to_images.values()):
            break
    x_train_sampled = []
    y_train_sampled = []
    for label in range(10):
        x_train_sampled.extend(label_to_images[label])
        y_train_sampled.extend([label] * num_samples)
    return np.array(x_train_sampled), np.array(y_train_sampled)
    
# Prediction logic
def predict_label(test_img, x_train_small, y_train_small):
    scores = defaultdict(list)
    for train_img, label in zip(x_train_small, y_train_small):
        match = np.sum(test_img == train_img)
        scores[label].append(match / 784.0)
    avg_scores = {label: np.max(score_list) for label, score_list in scores.items()}
    return max(avg_scores, key=avg_scores.get)
    
# Wrap predict_label to allow use in executor (requires args to be passed)
def predict_label_wrapper(args):
    test_img, x_train_small, y_train_small = args
    return predict_label(test_img, x_train_small, y_train_small)

args_list = [(img, x_train_small, y_train_small) for img in x_test_enc]
print("Date * Time: ", datetime.datetime.now(), "\nTimestamp: ", datetime.datetime.now())
# Use 4 workers for parallel prediction
with ProcessPoolExecutor(max_workers=4) as executor:
    preds = list(tqdm(executor.map(predict_label_wrapper, args_list), total=len(x_test_enc), desc="Predicting for 10000 MNIST encrypted-permuted Test images...."))
print("Date * Time: ", datetime.datetime.now(), "\nTimestamp: ", datetime.datetime.now())

acc = np.mean(np.array(preds) == y_test) * 100
print(f"Accuracy: {acc:.2f}%")

Date * Time:  2025-08-07 09:33:07.041417 
Timestamp:  2025-08-07 09:33:07.041421


Predicting for 10000 MNIST encrypted-permuted Test images....: 100%|██████████| 10000/10000 [1:06:31<00:00,  2.51it/s]

Date * Time:  2025-08-07 10:39:39.981521 
Timestamp:  2025-08-07 10:39:39.981526
Accuracy: 33.05%
